# 🔍 AIID Policy & Research Visualizations
### AI Incident Database — Master Dataset Explorer

This notebook produces **3 high-impact visualizations** designed for:
- **Policy makers** — identifying escalating risk areas and sectors needing regulation
- **Researchers** — spotting temporal trends, harm patterns, and critical sectors

---

## 👋 How to use this notebook

**3 simple steps:**
1. Click **Runtime → Run all** (or press `Ctrl+F9`)
2. When prompted in **Cell 3**, click **Choose Files** and upload your `AIID_Master_Dataset.xlsx`
3. Wait ~30 seconds — all 3 charts will appear automatically

**To re-run with a different file:** Click Runtime → Run all again and upload the new file when prompted.

---
### The 3 Visualizations
| # | Chart | Policy Question |
|---|-------|-----------------|
| 1 | **Risk Domain Escalation Heatmap** | Which AI risk categories are accelerating year-over-year? |
| 2 | **Sector Harm Profile** | Which industries face the highest real-world harm exposure? |
| 3 | **Incident Surge vs. Harm Timeline** | Is the explosion in incidents translating into more real harm? |

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  ⚙️  CELL 1 — CONFIGURATION                                                 ║
# ║  This is the only cell you ever need to edit.                               ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ── 1A. EXCEL SHEET NAME ──────────────────────────────────────────────────────
# The name of the sheet containing the master data inside your Excel file.
# Default matches the AIID Master Dataset Builder output.
SHEET_NAME = "Master Dataset"

# ── 1B. HEADER ROW ────────────────────────────────────────────────────────────
# Which row in the Excel sheet contains the column names.
# The AIID Master Dataset Builder uses row 3 (rows 1-2 are title/colour bands).
# If you're using a plain CSV-style file with headers in row 1, set this to 1.
HEADER_ROW = 3

# ── 1C. COLUMN NAMES ──────────────────────────────────────────────────────────
# Maps what the notebook expects → the actual column name in your Excel file.
# If the Schema Health Check flags a column as missing, update the VALUE here.
COLUMN_MAP = {
    "incident_id"   : "Incident ID",
    "year"          : "year",
    "title"         : "title",
    "risk_domain"   : "Risk Domain",
    "risk_subdomain": "Risk Subdomain",
    "intent"        : "Intent",
    "tangible_harm" : "Tangible Harm",
    "sector"        : "Sector of Deployment",
    "lives_lost"    : "Lives Lost",
    "injuries"      : "Injuries",
    "deployer"      : "deployer",
    "data_sources"  : "Data Sources",
}

# ── 1D. VISUALIZATION SETTINGS ────────────────────────────────────────────────
VIZ_YEAR_START = 2015       # First year to show on charts
VIZ_YEAR_END   = 2025       # Last year to show on charts
SAVE_CHARTS    = True       # Save PNGs alongside the Excel file
CHART_DPI      = 150        # Resolution of saved PNGs

# ── 1E. SECTOR GROUPING RULES ─────────────────────────────────────────────────
# Maps keywords found in 'Sector of Deployment' → clean display label.
# Add new rows if your data uses different sector names.
SECTOR_RULES = [
    (["transport"],                          "Transportation"),
    (["health"],                             "Healthcare"),
    (["law enforce", "police", "justice"],   "Law Enforcement"),
    (["education", "school", "university"],  "Education"),
    (["financ", "bank", "insurance"],        "Finance & Banking"),
    (["retail", "wholesale", "commerce"],    "Retail & Commerce"),
    (["admin", "public admin", "government"],"Government & Admin"),
    (["information", "arts", "media"],       "Tech & Media"),
    (["manufactur", "mining", "industrial"], "Industry & Manufacturing"),
    (["agriculture", "farming"],             "Agriculture"),
    (["military", "defence", "defense"],     "Defence"),
]

print("✅  Configuration loaded")
print(f"    Sheet name    : {SHEET_NAME}")
print(f"    Header row    : {HEADER_ROW}")
print(f"    Year range    : {VIZ_YEAR_START} – {VIZ_YEAR_END}")
print(f"    Save charts   : {SAVE_CHARTS}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📦  CELL 2 — INSTALL & IMPORTS  (no editing needed)                        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

import subprocess, sys
for pkg in ["openpyxl", "pandas", "matplotlib", "seaborn", "numpy"]:
    subprocess.run([sys.executable, "-m", "pip", "install", pkg, "-q"], check=True)

import warnings
warnings.filterwarnings("ignore")

import io, os
from pathlib import Path
from collections import defaultdict

import numpy  as np
import pandas as pd
import matplotlib.pyplot       as plt
import matplotlib.ticker       as mticker
import matplotlib.patheffects  as pe
from matplotlib.colors       import LinearSegmentedColormap
from matplotlib.gridspec     import GridSpec
from matplotlib.patches      import Rectangle, FancyArrowPatch

# Google Colab helpers
try:
    from google.colab import files as colab_files
    IN_COLAB = True
except ImportError:
    colab_files = None
    IN_COLAB = False

# ── Global dark theme ─────────────────────────────────────────────────────────
plt.rcParams.update({
    "figure.facecolor" : "#0d1117",
    "axes.facecolor"   : "#161b22",
    "axes.edgecolor"   : "#30363d",
    "axes.labelcolor"  : "#c9d1d9",
    "axes.titlecolor"  : "#e6edf3",
    "axes.titlesize"   : 13,
    "axes.titleweight" : "bold",
    "axes.labelsize"   : 10,
    "xtick.color"      : "#8b949e",
    "ytick.color"      : "#8b949e",
    "xtick.labelsize"  : 8.5,
    "ytick.labelsize"  : 8.5,
    "grid.color"       : "#21262d",
    "grid.linewidth"   : 0.6,
    "text.color"       : "#c9d1d9",
    "font.family"      : "monospace",
    "legend.facecolor" : "#161b22",
    "legend.edgecolor" : "#30363d",
    "legend.fontsize"  : 8.5,
})

ACCENT  = "#58a6ff"
DANGER  = "#f85149"
WARN    = "#e3b341"
SUCCESS = "#3fb950"
PURPLE  = "#bc8cff"

print(f"✅  Libraries ready  |  Running in {'Google Colab' if IN_COLAB else 'local Jupyter'}")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📁  CELL 3 — UPLOAD YOUR FILE  (no editing needed)                         ║
# ║                                                                              ║
# ║  In Google Colab: a file picker will appear — click "Choose Files" and      ║
# ║  select your AIID_Master_Dataset.xlsx file.                                 ║
# ║                                                                              ║
# ║  Running locally: place your Excel file in the same folder as this          ║
# ║  notebook and set LOCAL_FILEPATH below.                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ── For local Jupyter only: set the path to your Excel file here ─────────────
LOCAL_FILEPATH = "AIID_Master_Dataset.xlsx"   # ← change this if running locally

UPLOADED_PATH = None

if IN_COLAB:
    print("📂  Please select your AIID Master Dataset Excel file...")
    print("    (Click the 'Choose Files' button that appears below)")
    print()
    uploaded = colab_files.upload()

    if not uploaded:
        raise ValueError(
            "No file was uploaded. Please re-run this cell and select your Excel file."
        )

    filename = list(uploaded.keys())[0]
    if not filename.endswith((".xlsx", ".xls")):
        raise ValueError(
            f"Uploaded file '{filename}' does not appear to be an Excel file (.xlsx/.xls). "
            "Please upload the correct file."
        )

    UPLOADED_PATH = f"/content/{filename}"
    print(f"\n✅  File uploaded : {filename}")
    print(f"    Size          : {len(uploaded[filename]) / 1024:.1f} KB")

else:
    # Local mode — check file exists
    p = Path(LOCAL_FILEPATH)
    if not p.exists():
        raise FileNotFoundError(
            f"File not found: {p.resolve()}\n"
            f"Please place your Excel file at that path, or update LOCAL_FILEPATH above."
        )
    UPLOADED_PATH = str(p)
    print(f"✅  File found    : {p.resolve()}")
    print(f"    Size          : {p.stat().st_size / 1024:.1f} KB")

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🔍  CELL 4 — SCHEMA HEALTH CHECK  (no editing needed)                      ║
# ║  Checks that all expected columns exist before loading data.                ║
# ║  If a column is flagged MISSING, update COLUMN_MAP in Cell 1.               ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

SEP = "─" * 60

# ── Read only headers from the file (fast) ────────────────────────────────────
try:
    _preview = pd.read_excel(
        UPLOADED_PATH,
        sheet_name = SHEET_NAME,
        header     = HEADER_ROW - 1,  # pandas is 0-indexed
        nrows      = 0
    )
    actual_cols = set(_preview.columns.astype(str))
except Exception as e:
    # Provide a clear error if sheet name is wrong
    import openpyxl as _opx
    _wb = _opx.load_workbook(UPLOADED_PATH, read_only=True)
    available_sheets = _wb.sheetnames
    raise ValueError(
        f"Could not find sheet '{SHEET_NAME}' in the uploaded file.\n"
        f"Sheets available: {available_sheets}\n"
        f"Update SHEET_NAME in Cell 1 to match one of the above."
    ) from e

# ── Compare COLUMN_MAP against actual columns ─────────────────────────────────
print(SEP)
print("  🔍  SCHEMA HEALTH CHECK")
print(SEP)
print(f"  Sheet  : '{SHEET_NAME}'")
print(f"  Columns found in file: {len(actual_cols)}")
print()

all_ok  = True
missing = []

for key, col_name in COLUMN_MAP.items():
    found = col_name in actual_cols
    status = "✅" if found else "❌"
    if not found:
        all_ok = False
        missing.append((key, col_name))
        # Suggest close matches
        close = [
            c for c in actual_cols
            if col_name.lower().replace(" ","") in c.lower().replace(" ","")
            or c.lower().replace(" ","") in col_name.lower().replace(" ","")
        ]
        hint = f"  → Did you mean: {close[:3]}?" if close else ""
        print(f"  {status}  '{col_name}'{hint}")
    else:
        print(f"  {status}  '{col_name}'")

print()
print(SEP)
if all_ok:
    print("  ✅  All columns found — safe to load data")
else:
    print(f"  ❌  {len(missing)} column(s) not found")
    print()
    print("  HOW TO FIX:")
    print("  1. Go back to Cell 1 (⚙️ Configuration)")
    print("  2. In COLUMN_MAP, find the entry for the missing column")
    print("  3. Update the VALUE to match the exact column name shown above")
    print("  4. Re-run from Cell 1")
print(SEP)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📂  CELL 5 — LOAD DATA  (no editing needed)                                ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

print("Loading data ... ", end="", flush=True)

df_raw = pd.read_excel(
    UPLOADED_PATH,
    sheet_name = SHEET_NAME,
    header     = HEADER_ROW - 1,
    dtype      = str               # read everything as strings first — safe for mixed types
)

# Rename to internal keys using COLUMN_MAP (only rename columns that exist)
rename_map = {v: k for k, v in COLUMN_MAP.items() if v in df_raw.columns}
df = df_raw.rename(columns=rename_map).copy()

# Drop rows with no Incident ID (blank rows from Excel formatting)
df = df[df["incident_id"].notna() & (df["incident_id"].str.strip() != "")].copy()

# Parse year to int
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

# Parse numeric columns
for col in ["lives_lost", "injuries"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# Normalise text columns (strip whitespace)
for col in ["risk_domain", "risk_subdomain", "intent", "tangible_harm", "sector"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace("nan", "")

print(f"done")

# ── Summary ───────────────────────────────────────────────────────────────────
total_rows  = len(df)
year_min    = int(df["year"].dropna().min())
year_max    = int(df["year"].dropna().max())
total_cols  = len(df.columns)

mit_pct  = (df["risk_domain"].replace("", pd.NA).notna().mean() * 100) if "risk_domain" in df.columns else 0
harm_pct = (df["tangible_harm"].replace("", pd.NA).notna().mean() * 100) if "tangible_harm" in df.columns else 0
sec_pct  = (df["sector"].replace("", pd.NA).notna().mean() * 100) if "sector" in df.columns else 0

SEP = "═" * 52
print(f"\n{SEP}")
print(f"  DATASET LOADED SUCCESSFULLY")
print(SEP)
print(f"  Total incidents        : {total_rows:>7,}")
print(f"  Date range             : {year_min} → {year_max}")
print(f"  Columns                : {total_cols}")
print()
print(f"  Column coverage:")
print(f"    Risk Domain (MIT)    : {mit_pct:>5.1f}%")
print(f"    Tangible Harm (CSET) : {harm_pct:>5.1f}%")
print(f"    Sector (CSET)        : {sec_pct:>5.1f}%")

# ── Auto-detect Risk Domains from data (replaces hardcoded list) ──────────────
YEARS = list(range(VIZ_YEAR_START, VIZ_YEAR_END + 1))

if "risk_domain" in df.columns:
    DOMAINS = (
        df[df["year"].isin(YEARS)]["risk_domain"]
        .replace("", pd.NA).dropna()
        .value_counts()
        .index.tolist()
    )
    print(f"\n  Risk Domains detected  : {len(DOMAINS)}")
    for d in DOMAINS:
        print(f"    • {d}")
else:
    DOMAINS = []

print(SEP)

---
## 📊 Visualization 1 — Risk Domain Escalation Heatmap
**Policy Question:** *Which AI risk categories are accelerating, and where should regulatory attention focus?*

Each cell shows the **raw incident count** for that risk category in that year.  
Colour intensity is **log-normalized per row** so both slow-burn and fast-escalating risks are visible side-by-side.  
The ⚠ Surge Zone highlights the most recent years where growth is sharpest.

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📊  CELL 6 — VISUALIZATION 1: Risk Domain Escalation Heatmap               ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

if not DOMAINS:
    print("⚠️  No Risk Domain data found in the dataset. Skipping Visualization 1.")
    print("   Check that 'risk_domain' is mapped correctly in Cell 1 (COLUMN_MAP).")
else:
    # ── Build domain × year matrix ────────────────────────────────────────────
    matrix = np.zeros((len(DOMAINS), len(YEARS)), dtype=int)
    viz1_df = df[df["year"].isin(YEARS) & df["risk_domain"].ne("")]

    for _, row in viz1_df.iterrows():
        yr  = row["year"]
        dom = row["risk_domain"]
        if yr in YEARS and dom in DOMAINS:
            matrix[DOMAINS.index(dom)][YEARS.index(yr)] += 1

    # Row-wise log-normalize for colour intensity
    matrix_norm = np.zeros_like(matrix, dtype=float)
    for i, row in enumerate(matrix):
        m = row.max()
        if m > 0:
            matrix_norm[i] = np.log1p(row) / np.log1p(m)

    # ── Dynamic insight computation ───────────────────────────────────────────
    # Find the domain with the highest growth in the last 3 years vs prior 3 years
    insights = []
    last_3_idx  = [YEARS.index(y) for y in YEARS[-3:]  if y in YEARS]
    prior_3_idx = [YEARS.index(y) for y in YEARS[-6:-3] if y in YEARS]

    for i, dom in enumerate(DOMAINS):
        recent = matrix[i][last_3_idx].sum()  if last_3_idx  else 0
        prior  = matrix[i][prior_3_idx].sum() if prior_3_idx else 0
        if prior > 0:
            growth_pct = (recent - prior) / prior * 100
        elif recent > 0:
            growth_pct = 999
        else:
            growth_pct = 0
        insights.append((dom, recent, prior, growth_pct))

    insights.sort(key=lambda x: x[3], reverse=True)
    domain_totals = matrix.sum(axis=1)

    # ── Wrap long domain labels ────────────────────────────────────────────────
    def wrap_label(label, max_len=22):
        if len(label) <= max_len:
            return label
        parts = label.split(" ")
        mid = len(parts) // 2
        return " ".join(parts[:mid]) + "\n" + " ".join(parts[mid:])

    domain_labels = [wrap_label(d) for d in DOMAINS]

    # ── Plot ──────────────────────────────────────────────────────────────────
    cmap = LinearSegmentedColormap.from_list(
        "aiid", ["#0d1117", "#1f4068", "#1b6ca8", "#e3b341", "#f85149"], N=256
    )

    fig, ax = plt.subplots(figsize=(16, max(5, len(DOMAINS) * 0.85 + 1.5)))
    fig.patch.set_facecolor("#0d1117")
    ax.set_facecolor("#0d1117")

    im = ax.imshow(matrix_norm, cmap=cmap, aspect="auto", vmin=0, vmax=1)

    for i in range(len(DOMAINS)):
        for j in range(len(YEARS)):
            val  = matrix[i][j]
            norm = matrix_norm[i][j]
            col  = "white" if norm > 0.4 else "#8b949e"
            fw   = "bold"  if norm > 0.65 else "normal"
            ax.text(j, i, str(val), ha="center", va="center",
                    fontsize=8.5, color=col, fontweight=fw, fontfamily="monospace")

    ax.set_xticks(range(len(YEARS)))
    ax.set_xticklabels(YEARS, fontsize=9)
    ax.set_yticks(range(len(domain_labels)))
    ax.set_yticklabels(domain_labels, fontsize=9)

    ax.set_title(
        f"Risk Domain Escalation Heatmap  ·  Incident Count by Category & Year ({VIZ_YEAR_START}–{VIZ_YEAR_END})\n"
        "Colour intensity = log-normalized within each row  |  Numbers = raw counts",
        pad=14, fontsize=12, color="#e6edf3"
    )

    cb = fig.colorbar(im, ax=ax, fraction=0.015, pad=0.02)
    cb.set_label("Normalized Intensity (per domain peak)", color="#8b949e", fontsize=8)
    cb.ax.yaxis.set_tick_params(color="#8b949e")
    cb.outline.set_edgecolor("#30363d")
    plt.setp(cb.ax.yaxis.get_ticklabels(), color="#8b949e")

    # Right-side totals
    ax2 = ax.twinx()
    ax2.set_ylim(-0.5, len(DOMAINS) - 0.5)
    ax2.set_yticks(range(len(DOMAINS)))
    ax2.set_yticklabels([f"  Σ {t:,}" for t in domain_totals], fontsize=8, color=ACCENT)
    ax2.tick_params(length=0)
    ax2.spines[:].set_visible(False)

    # Surge zone box — last 3 years
    surge_start = len(YEARS) - 3 - 0.5
    rect = Rectangle((surge_start, -0.5), 3, len(DOMAINS),
                      linewidth=1.5, edgecolor=WARN, facecolor="none",
                      linestyle="--", alpha=0.6)
    ax.add_patch(rect)
    ax.text(surge_start + 1.5, -0.85, "⚠ Surge Zone",
            ha="center", fontsize=8, color=WARN, fontfamily="monospace")

    ax.spines[:].set_edgecolor("#30363d")
    plt.tight_layout()

    if SAVE_CHARTS:
        out_path = str(Path(UPLOADED_PATH).parent / "viz1_risk_heatmap.png")
        plt.savefig(out_path, dpi=CHART_DPI, bbox_inches="tight",
                    facecolor="#0d1117", edgecolor="none")
        print(f"💾  Saved: {out_path}")

    plt.show()

    # ── Dynamic insights ──────────────────────────────────────────────────────
    print("\n📌  KEY INSIGHTS — computed from your data")
    print(f"    Total incidents {VIZ_YEAR_START}–{VIZ_YEAR_END}: {matrix.sum():,}")
    print(f"    Fastest-growing risk domains (last 3 years vs prior 3 years):")
    for dom, recent, prior, pct in insights[:3]:
        if pct == 999:
            print(f"      • {dom:<45}  New category — {recent} incidents")
        else:
            direction = "▲" if pct > 0 else "▼"
            print(f"      • {dom:<45}  {direction} {abs(pct):+.0f}%  ({prior} → {recent} incidents)")
    print(f"    Largest domain by total volume: {DOMAINS[domain_totals.argmax()]} ({domain_totals.max():,} incidents)")

---
## 📊 Visualization 2 — Sector Harm Profile
**Policy Question:** *Which sectors are most exposed to real, tangible AI-caused harm — and should be prioritized for sector-specific AI regulation?*

This stacked horizontal bar chart shows **harm severity** (Confirmed → No Harm) broken down by **industry sector**.  
The **Harm Risk Score** (weighted composite: Confirmed×4 + Near Miss×2 + Issue×1) ranks sectors by regulatory urgency.

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📊  CELL 7 — VISUALIZATION 2: Sector Harm Profile                          ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

if "sector" not in df.columns or "tangible_harm" not in df.columns:
    print("⚠️  Sector or Tangible Harm column not found. Skipping Visualization 2.")
    print("   Check COLUMN_MAP in Cell 1.")
else:
    # ── Apply sector grouping rules from Cell 1 config ────────────────────────
    def classify_sector(raw):
        if not raw or raw == "nan":
            return None
        s = str(raw).lower()
        for keywords, label in SECTOR_RULES:
            if any(kw in s for kw in keywords):
                return label
        return None

    HARM_CATS  = ["Confirmed Harm", "Near Miss", "Risk Issue", "No Harm / Unclear"]
    HARM_SCORE = {"Confirmed Harm": 4, "Near Miss": 2, "Risk Issue": 1, "No Harm / Unclear": 0}

    def classify_harm(raw):
        s = str(raw or "").lower()
        if "definitively" in s: return "Confirmed Harm"
        if "near miss"    in s: return "Near Miss"
        if "issue"        in s: return "Risk Issue"
        return "No Harm / Unclear"

    sector_counts = defaultdict(lambda: defaultdict(int))
    for _, row in df.iterrows():
        sec  = classify_sector(row.get("sector"))
        if not sec:
            continue
        harm = classify_harm(row.get("tangible_harm"))
        sector_counts[sec][harm] += 1

    if not sector_counts:
        print("⚠️  No sector data could be classified. Check your SECTOR_RULES in Cell 1.")
    else:
        # Sort by Harm Risk Score ascending (highest ends up at top of horiz chart)
        sectors = sorted(
            sector_counts.keys(),
            key=lambda s: sum(sector_counts[s][c] * HARM_SCORE[c] for c in HARM_CATS)
        )
        scores     = [sum(sector_counts[s][c] * HARM_SCORE[c] for c in HARM_CATS) for s in sectors]
        totals_sec = [sum(sector_counts[s][c] for c in HARM_CATS) for s in sectors]

        BAR_COLORS = {
            "Confirmed Harm"    : DANGER,
            "Near Miss"         : WARN,
            "Risk Issue"        : ACCENT,
            "No Harm / Unclear" : "#484f58",
        }

        fig, (ax_main, ax_score) = plt.subplots(
            1, 2, figsize=(16, max(5, len(sectors) * 0.7 + 2)),
            gridspec_kw={"width_ratios": [3.2, 1]}
        )
        fig.patch.set_facecolor("#0d1117")

        lefts = np.zeros(len(sectors))
        for cat in HARM_CATS:
            vals = [sector_counts[s][cat] for s in sectors]
            bars = ax_main.barh(sectors, vals, left=lefts,
                                color=BAR_COLORS[cat], label=cat,
                                height=0.62, alpha=0.92)
            for bar, v in zip(bars, vals):
                if v >= 2:
                    x = bar.get_x() + bar.get_width() / 2
                    y = bar.get_y() + bar.get_height() / 2
                    ax_main.text(x, y, str(v), ha="center", va="center",
                                 fontsize=8, color="white",
                                 fontfamily="monospace", fontweight="bold")
            lefts += vals

        for i, (s, tot) in enumerate(zip(sectors, totals_sec)):
            ax_main.text(lefts[i] + 0.4, i, f" {tot}", va="center",
                         fontsize=8.5, color="#8b949e", fontfamily="monospace")

        ax_main.set_facecolor("#161b22")
        ax_main.spines[:].set_edgecolor("#30363d")
        ax_main.set_xlabel("Number of Incidents", labelpad=8)
        ax_main.set_title(
            "Sector Harm Profile  ·  Incident Severity by Industry\n"
            "Sorted by Harm Risk Score  (Confirmed×4 + Near Miss×2 + Issue×1)",
            fontsize=12, pad=12
        )
        ax_main.legend(loc="lower right", framealpha=0.9)
        ax_main.grid(axis="x", alpha=0.3)
        ax_main.set_axisbelow(True)

        score_colors = [
            DANGER if sc >= 50 else WARN if sc >= 20 else ACCENT
            for sc in scores
        ]
        bars_s = ax_score.barh(sectors, scores, color=score_colors, height=0.62, alpha=0.88)
        for bar, sc in zip(bars_s, scores):
            ax_score.text(
                bar.get_width() + 0.5, bar.get_y() + bar.get_height() / 2,
                f"{sc:.0f}", va="center", fontsize=8.5, color="#c9d1d9",
                fontfamily="monospace", fontweight="bold"
            )

        ax_score.set_facecolor("#161b22")
        ax_score.spines[:].set_edgecolor("#30363d")
        ax_score.set_xlabel("Harm Risk Score", labelpad=8)
        ax_score.set_title("Risk\nScore", fontsize=10, pad=12)
        ax_score.set_yticks([])
        ax_score.grid(axis="x", alpha=0.3)
        ax_score.set_axisbelow(True)

        if max(scores) > 0:
            high_urgency = max(scores) * 0.6
            ax_score.axvline(high_urgency, color=DANGER, linestyle="--", alpha=0.5, linewidth=1)
            ax_score.text(high_urgency + 0.5, -0.6, "High\nUrgency",
                          color=DANGER, fontsize=7, va="bottom")

        plt.tight_layout()

        if SAVE_CHARTS:
            out_path = str(Path(UPLOADED_PATH).parent / "viz2_sector_harm.png")
            plt.savefig(out_path, dpi=CHART_DPI, bbox_inches="tight",
                        facecolor="#0d1117", edgecolor="none")
            print(f"💾  Saved: {out_path}")

        plt.show()

        # ── Dynamic insights ──────────────────────────────────────────────────
        sorted_by_score = sorted(zip(sectors, scores), key=lambda x: x[1], reverse=True)
        print("\n📌  KEY INSIGHTS — computed from your data")
        print(f"    Sectors analysed: {len(sectors)}")
        print(f"    Top sectors by Harm Risk Score:")
        for rank, (sec, score) in enumerate(sorted_by_score[:5], 1):
            confirmed = sector_counts[sec]["Confirmed Harm"]
            total_s   = sum(sector_counts[sec][c] for c in HARM_CATS)
            pct       = confirmed / total_s * 100 if total_s > 0 else 0
            print(f"      #{rank}  {sec:<30}  Score: {score:>4.0f}  |  "
                  f"Confirmed harm: {confirmed} ({pct:.0f}% of sector incidents)")

---
## 📊 Visualization 3 — Incident Surge vs. Harm Severity Over Time
**Policy Question:** *Is the explosive growth in AI incidents translating into more real-world harm — or are safeguards keeping pace?*

This dual-axis chart overlays:
- **Total incident volume** (bars) split by Intentional vs Unintentional  
- **Confirmed harm rate** (%) as a dashed line — showing whether harm keeps pace with incident growth  
- **Injury counts** as a secondary panel — the physical impact signal

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📊  CELL 8 — VISUALIZATION 3: Incident Surge vs. Harm Timeline             ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ── Build year-level summary ──────────────────────────────────────────────────
year_stats = {
    y: {"total": 0, "intentional": 0, "unintentional": 0,
        "harm_events": 0, "near_miss": 0, "lives": 0.0, "injuries": 0.0}
    for y in YEARS
}

for _, row in df.iterrows():
    yr = row.get("year")
    if pd.isna(yr) or int(yr) not in YEARS:
        continue
    st = year_stats[int(yr)]
    st["total"] += 1
    intent = str(row.get("intent") or "").lower()
    if   "intentional"   == intent: st["intentional"]   += 1
    elif "unintentional" == intent: st["unintentional"] += 1
    harm = str(row.get("tangible_harm") or "").lower()
    if "definitively" in harm: st["harm_events"] += 1
    if "near miss"    in harm: st["near_miss"]   += 1
    st["injuries"] += float(row.get("injuries") or 0)
    st["lives"]    += float(row.get("lives_lost") or 0)

totals_v3  = [year_stats[y]["total"]         for y in YEARS]
intents_v3 = [year_stats[y]["intentional"]   for y in YEARS]
uninten_v3 = [year_stats[y]["unintentional"] for y in YEARS]
harms_v3   = [year_stats[y]["harm_events"]   for y in YEARS]
inj_raw    = [year_stats[y]["injuries"]       for y in YEARS]
inj_cap    = 1500  # cap for display scale
injuries_v3 = [min(v, inj_cap) for v in inj_raw]
harm_rate  = [h / t * 100 if t > 0 else 0 for h, t in zip(harms_v3, totals_v3)]

# ── Plot ──────────────────────────────────────────────────────────────────────
fig = plt.figure(figsize=(16, 8), facecolor="#0d1117")
gs  = GridSpec(3, 1, height_ratios=[3, 1, 0.08], hspace=0.08)
ax_main = fig.add_subplot(gs[0])
ax_inj  = fig.add_subplot(gs[1], sharex=ax_main)
ax_gap  = fig.add_subplot(gs[2])
ax_gap.axis("off")

x = np.arange(len(YEARS))
w = 0.65

# Stacked bars
ax_main.bar(x, uninten_v3, width=w, label="Unintentional",
            color="#1f4068", alpha=0.9, zorder=2)
ax_main.bar(x, intents_v3, width=w, bottom=uninten_v3,
            label="Intentional", color=DANGER, alpha=0.85, zorder=2)

# Harm event bubbles
y_bubbles = [u + i + max(totals_v3) * 0.04 for u, i in zip(uninten_v3, intents_v3)]
ax_main.scatter(x, y_bubbles,
                s=[h * 18 + 10 for h in harms_v3],
                color=WARN, alpha=0.85, zorder=3,
                label="Confirmed Harm (bubble size)",
                edgecolors="white", linewidths=0.5)

# Harm rate line (twin axis)
ax_r = ax_main.twinx()
ax_r.set_facecolor("none")
ax_r.plot(x, harm_rate, color=WARN, linewidth=2, linestyle="--",
          marker="o", markersize=5, zorder=4, label="Harm Rate %")
ax_r.fill_between(x, harm_rate, alpha=0.08, color=WARN)
ax_r.set_ylabel("Confirmed Harm Rate (%)", color=WARN, fontsize=9)
ax_r.tick_params(axis="y", colors=WARN, labelsize=8)
ax_r.spines[:].set_edgecolor("#30363d")
if max(harm_rate) > 0:
    ax_r.set_ylim(0, max(harm_rate) * 2.2)
ax_r.yaxis.set_major_formatter(mticker.FormatStrFormatter("%.0f%%"))

# Total labels on bars
for xi, tot in enumerate(totals_v3):
    if tot > 0:
        ax_main.text(xi, tot + max(totals_v3) * 0.015, str(tot),
                     ha="center", fontsize=7.5, color="#c9d1d9",
                     fontfamily="monospace", fontweight="bold")

# ── Annotate significant real-world events dynamically ────────────────────────
# Find the 2 years with the biggest YoY jumps and annotate them
yoy_jumps = [(YEARS[i], totals_v3[i] - totals_v3[i-1])
             for i in range(1, len(YEARS)) if totals_v3[i-1] > 0]
top_jumps = sorted(yoy_jumps, key=lambda x: x[1], reverse=True)[:2]

for yr, jump in top_jumps:
    xi  = YEARS.index(yr)
    pct = jump / totals_v3[xi-1] * 100 if totals_v3[xi-1] > 0 else 0
    ax_main.annotate(
        f"+{pct:.0f}% YoY",
        xy=(xi, totals_v3[xi]),
        xytext=(xi + 0.35, totals_v3[xi] + max(totals_v3) * 0.1),
        fontsize=7.5, color="#8b949e",
        arrowprops=dict(arrowstyle="->", color="#484f58", lw=0.8),
        ha="left", va="bottom", fontfamily="monospace"
    )

ax_main.set_facecolor("#161b22")
ax_main.spines[:].set_edgecolor("#30363d")
ax_main.set_ylabel("Number of Incidents", labelpad=8)
ax_main.set_xticks([])
ax_main.set_xlim(-0.5, len(YEARS) - 0.5)
ax_main.set_title(
    f"Incident Surge vs. Harm Severity  ·  {VIZ_YEAR_START}–{VIZ_YEAR_END}\n"
    "Bars = total incidents (Unintentional + Intentional)  |  Dashed line = confirmed harm rate  |  Bubbles = harm event count",
    fontsize=12, pad=12
)
ax_main.grid(axis="y", alpha=0.25, zorder=0)
ax_main.set_axisbelow(True)

h1, l1 = ax_main.get_legend_handles_labels()
h2, l2 = ax_r.get_legend_handles_labels()
ax_main.legend(h1 + h2, l1 + l2, loc="upper left", framealpha=0.9, ncol=2)

# Injury sub-panel
inj_colors = [DANGER if v > inj_cap * 0.5 else WARN if v > 50 else ACCENT
              for v in injuries_v3]
ax_inj.bar(x, injuries_v3, width=w, color=inj_colors, alpha=0.8, zorder=2)
for xi, (yr, v) in enumerate(zip(YEARS, injuries_v3)):
    actual = inj_raw[xi]
    label  = f"{actual/1000:.0f}k" if actual >= 1000 else str(int(actual))
    if actual > 0:
        ax_inj.text(xi, v + 15, label, ha="center", fontsize=7,
                    color="#c9d1d9", fontfamily="monospace")

ax_inj.set_facecolor("#161b22")
ax_inj.spines[:].set_edgecolor("#30363d")
ax_inj.set_ylabel(f"Injuries\n(cap {inj_cap/1000:.1f}k)", fontsize=8, labelpad=4)
ax_inj.set_xticks(x)
ax_inj.set_xticklabels(YEARS, fontsize=9)
ax_inj.grid(axis="y", alpha=0.25, zorder=0)
ax_inj.set_axisbelow(True)
ax_inj.set_xlim(-0.5, len(YEARS) - 0.5)

if SAVE_CHARTS:
    out_path = str(Path(UPLOADED_PATH).parent / "viz3_surge_vs_harm.png")
    plt.savefig(out_path, dpi=CHART_DPI, bbox_inches="tight",
                facecolor="#0d1117", edgecolor="none")
    print(f"💾  Saved: {out_path}")

plt.show()

# ── Dynamic insights ──────────────────────────────────────────────────────────
yr_max_vol  = YEARS[totals_v3.index(max(totals_v3))]
yr_max_harm = YEARS[harms_v3.index(max(harms_v3))] if max(harms_v3) > 0 else "N/A"
yr_first    = YEARS[0];  yr_last = YEARS[-1]
growth_mult = totals_v3[-1] / totals_v3[0] if totals_v3[0] > 0 else 0
avg_harm_rate_early = sum(harm_rate[:3]) / 3 if len(harm_rate) >= 3 else 0
avg_harm_rate_late  = sum(harm_rate[-3:]) / 3 if len(harm_rate) >= 3 else 0

print("\n📌  KEY INSIGHTS — computed from your data")
print(f"    Incident volume {yr_first}→{yr_last}: {totals_v3[0]} → {totals_v3[-1]}  "
      f"({growth_mult:.1f}× growth)")
print(f"    Year with most incidents : {yr_max_vol}  ({max(totals_v3):,})")
print(f"    Year with most confirmed harm events : {yr_max_harm}  ({max(harms_v3)})")
if avg_harm_rate_early > 0:
    harm_trend = "declining ↓" if avg_harm_rate_late < avg_harm_rate_early else "rising ↑"
    print(f"    Harm rate trend (early {yr_first}-{YEARS[2]} avg {avg_harm_rate_early:.1f}%  "
          f"→  late {YEARS[-3]}-{yr_last} avg {avg_harm_rate_late:.1f}%) : {harm_trend}")
total_intent_all = sum(intents_v3)
total_all        = sum(totals_v3)
if total_all > 0:
    print(f"    Intentional incidents overall : {total_intent_all:,}  "
          f"({total_intent_all/total_all*100:.1f}% of all incidents in this window)")

---
## 📝 Summary & Downloads

All 3 visualizations are complete. The charts have been:
- **Displayed above** in the notebook output
- **Saved as PNG files** alongside your uploaded Excel file (if `SAVE_CHARTS = True`)

To download the saved PNGs in Google Colab, run the cell below.

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  💾  CELL 9 — DOWNLOAD SAVED CHARTS  (Google Colab only)                    ║
# ║  Run this cell to download the PNG files to your computer.                  ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

chart_files = [
    str(Path(UPLOADED_PATH).parent / f)
    for f in ["viz1_risk_heatmap.png", "viz2_sector_harm.png", "viz3_surge_vs_harm.png"]
]

found = [f for f in chart_files if Path(f).exists()]

if not found:
    print("⚠️  No saved chart files found.")
    print("   Make sure SAVE_CHARTS = True in Cell 1 and re-run the visualization cells.")
elif IN_COLAB:
    print("📥  Downloading charts to your computer...")
    for f in found:
        colab_files.download(f)
        print(f"    ✅  {Path(f).name}")
else:
    print("📁  Charts saved locally:")
    for f in found:
        print(f"    ✅  {Path(f).resolve()}")